### Random Forests and Photo-zs

In this notebook, we use Random Forests to estimate photometric redshifts starting from observations of galaxy magnitudes in six different photometric bands (u, g, r, i, z, y). 

Copyright: Viviana Acquaviva (2023); see also other data credits below.

License: [BSD-3-clause](https://opensource.org/license/bsd-3-clause/)

Essentially, we try to reproduce/improve upon the results of [this paper](https://arxiv.org/abs/1903.08174), for which the data are public and available [here](http://d-scholarship.pitt.edu/36064/). Additionally, we are very grateful to Jeff Newman for his expert advice.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)


font = {'size'   : 16}
matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
matplotlib.rcParams.update({'figure.autolayout': False})
matplotlib.rcParams['figure.dpi'] = 300

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_validate, KFold, cross_val_predict, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [ ]:
import astropy

from astropy.io import fits

#fits stands for Flexible Image Transport System; it's a format that allows one to store images and summary data

#### Data import

We can read the data into a data frame using pandas:

In [ ]:
with fits.open('DEEP2_uniq_Terapix_Subaru_v1.fits') as data:
    df = pd.DataFrame(np.array(data[1].data).byteswap().newbyteorder()) #see https://numpy.org/devdocs/user/basics.byteswapping.html#changing-byte-ordering

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.shape

We can select the columns we want, corresponding to the brightness of the galaxies in the six bands of interest.

In [ ]:
features = df[['u_apercor', 'g_apercor', 'r_apercor', 'i_apercor', 'z_apercor','y_apercor']]

The target property is the redshift. For this catalog, spectroscopic (more precise) redshifts, to use as the ground truth, are available in the column "zhelio".

In [ ]:
target = df['zhelio']

In [ ]:
features.head(10)

In [ ]:
target.head(10)

### Ok, we are now ready to run our first Random Forest model!

To get an idea of what we are shooting for, we can look at a figure in the paper:

 ![Performance of photometric redshift reconstruction](Photoz_RF_CFHTLS_Deep.png)

In the figure above, $\sigma_{NMAD}$ is the normalized median absolute deviation of the residual vector, and $\eta$  is the fraction of outliers, defined as those objects for which (z_true - z_est)/(1+z_true) > 0.15.

To be fair, we are working with DEEP2/3 data, so our range is slightly different.

In [ ]:
model = RandomForestRegressor()

In [ ]:
model.get_params()

We begin by establishing a benchmark (note: it takes a little time; ~35 secs on my machine).

In [ ]:
scores = cross_validate(model, features, target, cv = KFold(n_splits=5, shuffle=True, random_state=10), return_train_score=True)

In [ ]:
scores

As a reminder, the scores are the R2 score at this point.

In [ ]:
np.mean(scores['test_score'])

In [ ]:
np.mean(scores['train_score'])

Looks like we have a severe ..... issue! Let's also check the predictions and plot them against the true values.

In [ ]:
ypred = cross_val_predict(model, features, target, \
            cv = KFold(n_splits=5, shuffle=True, random_state=10))

In [ ]:
plt.scatter(target,ypred, s = 20, c = 'royalblue')
plt.xlabel('True (spectroscopic) z', fontsize=14)
plt.ylabel('Predicted z',fontsize=14)
plt.axis('square')
plt.xlim(0,3)
plt.ylim(0,3)

### Question: Does it look like the one of the paper?

It's also interesting to look at the distribution of the predicted values, to see how they always tend to produce a narrower distribution. Why?

In [ ]:
plt.hist(target,bins=50,density=False,alpha=0.5, range = (0,3), label = 'True');
plt.hist(ypred,bins=50,density=False,alpha=0.5, range = (0,3), color = 'g', label = 'Predicted');
plt.legend(fontsize=14);

This is a characteristic of tree-based methods; examples with very low/high values of the target are "bunched up" with others that have higher/lower values, so the predictions tend to be over-estimated for low values of the target variables and under-estimated for high values. You can usually see this trend in scatter plots as well! 

Ok, we are now ready to calculate the outlier fraction:

In [ ]:
len(np.where(np.abs(target-ypred)>0.15*(1+target))[0])/len(target)

And NMAD

In [ ]:
1.48*np.median(np.abs(target-ypred)/(1 + target)) 
#The 1.48 is there because for a Gaussian distribution, this becomes the standard deviation

### Clearly, we are very far from the paper's performance :( 

### Because we have a pretty severe high variance issue, we can perform some parameter optimization.

We can start by making the data set a bit smaller, as we have seen that timings were already challenging in simple k-fold CV.

In [ ]:
np.random.seed(20)
sel = np.random.choice(range(len(ypred)), 5000, replace = False) #sample without replacement

In [ ]:
len(np.unique(sel))

And we create our new smaller data set.

In [ ]:
seld = features.loc[sel,:]
selt = target[sel]

It is good practice to ensure that the performance on the smaller set remains similar to the one obtained on the entire data set, which means that the change in size will not significantly affect the optimization process.

In [ ]:
littlescores = cross_validate(model,seld,selt, cv = KFold(n_splits=5, shuffle=True, random_state=10), return_train_score=True)

In [ ]:
littlescores['test_score'].mean(), littlescores['train_score'].mean()

### We are now ready to optimize hyperparameters - what should we choose?



#### Tree Parameters

Some useful parameters associated to a tree are:

-  The minimum number of instances in a leaf node;

-  The minimum number of instances required in a split node;

- The maximum depth of tree;

-  The criterion chosen to decide whether a split is "worth it", expressed in terms of information gain.


#### Randomization Parameters

Here we find:

- The number of k < n features that are used in building trees;

- The re-sampling (boostrap) of the data set (T or F).


#### Forest Parameters

The number of trees in the forest (n_estimators) can be adjusted, with the general understanding that more trees are better, but at some point performance will plateau, so one can find the trade-off between having more trees and lower runtime.

We can visualize them like this:

In [ ]:
model.get_params()

Here below is a possible set; we can run a grid search to look for the best model.

- min_impurity_decrease 

- number of trees

- max_leaf_nodes

- min_samples_split

- max_features

In [ ]:
#Takes a few minutes

parameters = {'min_impurity_decrease':[0.1, 0.5, 0.0], \
              'max_features':[None,4,2], 'n_estimators':[50, 100, 200], 'min_samples_split': [10,20,100], 
              'max_leaf_nodes':[None, 100, 200]}
nmodels = np.product([len(el) for el in parameters.values()])
model = GridSearchCV(RandomForestRegressor(), parameters, cv = KFold(n_splits=5, shuffle=True), \
                     verbose = 2, n_jobs = 4, return_train_score=True)
model.fit(seld,selt)

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

As usual, we save the results in a data frame, and look at the best models to build some intution.

In [ ]:
scores = pd.DataFrame(model.cv_results_)
scoresCV = scores[['params','mean_test_score','std_test_score','mean_train_score']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)


In [ ]:
scoresCV

### And the verdict is....

We have NOT improved the test scores.




















### Before giving up, and after noting that our issue is so sever that it's hard to attribute it to an optimization or chooice of algorithm issue, we need to look more carefully at data cleaning and/or imputing (something, in fact, that should be the first step in our pipeline!)

In my case, it was time to write to the authors of the paper, who told me exactly how they selected the data that went into making the learning set.

We needed to retain some additional columns to be used in the selection process.

In [ ]:
mags = df[['u_apercor', 'g_apercor', 'r_apercor', 'i_apercor', 'z_apercor','y_apercor','subaru_source','cfhtls_source','zquality']]

In [ ]:
mags.head()

In [ ]:
mags.shape

In [ ]:
#redshift quality - only use objects with high-quality spectroscopic redshift measurements

mags = mags[mags['zquality'] >= 3]

mags.shape

In [ ]:
#select objects with cfhtls deep photometric data 

mags = mags[mags['cfhtls_source'] == 0]

mags.shape

In [ ]:
#This additional selection (objects with subaru deep photometry) was irrelevant

#mags = mags[mags['subaru_source'] == 0]

#mags.shape

Unavailable measurements are marked by -99 or 99 (while typical values are around 20-25). We also get rid of data with missing measurements.

In [ ]:
mags = mags[mags > -10].dropna()

In [ ]:
mags.shape

In [ ]:
mags = mags[mags < 90].dropna()

In [ ]:
mags.shape

Our final set is made of the six original features and it has 6,307 objects.

In [ ]:
sel_features = mags[['u_apercor', 'g_apercor', 'r_apercor', 'i_apercor', 'z_apercor','y_apercor']]
sel_features.head()

We need, of course, to select the same set on the target vector.

In [ ]:
sel_target = target[sel_features.index]

Let's see how our benchmark model works. Note that for reproducible results we need to fix the random\_state parameter of the Random Forest (which controls the bootstrap process) and the random seed of the cross validation. 

In [ ]:
scores = cross_validate(RandomForestRegressor(random_state = 5),sel_features,sel_target,cv = KFold(n_splits=5, shuffle=True, random_state=10), \
               return_train_score=True)

In [ ]:
print(np.round(np.mean(scores['test_score']),3), np.round(np.std(scores['test_score']),3))

In [ ]:
print(np.round(np.mean(scores['train_score']),3), np.round(np.std(scores['train_score']),3))

The scores have improved noticeably! However, we can still observe high variance. We can re-run the optimization process (note that the data set size is limited, so we don't need to make it smaller).

In [ ]:
#This now takes ~3 minutes

parameters = {'max_depth':[3, 6, None], \
              'max_features':[None,4,2], 'n_estimators':[50,100,200], 'min_samples_leaf': [1,5,10]}
nmodels = np.product([len(el) for el in parameters.values()])
model = GridSearchCV(RandomForestRegressor(random_state = 5), parameters, cv = KFold(n_splits=5, shuffle=True, random_state=10), \
                     verbose = 2, n_jobs = 4, return_train_score=True)
model.fit(sel_features,sel_target)

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

In [ ]:
scores = pd.DataFrame(model.cv_results_)
scoresCV = scores[['params','mean_test_score','std_test_score','mean_train_score']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)
scoresCV

### Discussion Question: 

#### Do we need to explore some parameters more in detail (i.e., do we expect a significant improvement by enlarging the space of parameters?)



In [ ]:
bm = model.best_estimator_

We can generate one set of predictions to visualize what happens.

In [ ]:
ypred = cross_val_predict(bm, sel_features,sel_target, cv = KFold(n_splits=5, shuffle=True, random_state=10))

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(sel_target,ypred, s =10)
plt.xlabel('z_spec')
plt.ylabel('z_photo')
plt.ylim(0,2)
plt.xlim(0,2)

Calculate the outlier fraction:

In [ ]:
len(np.where(np.abs(sel_target-ypred)>0.15*(1+sel_target))[0])/len(sel_target)

Calculate Normalized Median Absolute Deviation (NMAD).

In [ ]:
1.48*np.median(np.abs(sel_target-ypred)/(1 + sel_target))

### Conclusion: How does our model compare with the paper's results?

### Can you think of a feature engineering exercise that can be helpful? (if you need a hint: it's in Section 7 of the paper ;) )







